## 子チャンク化, Section毎＋全体クエリ、re-rank

### 各種設定

In [1]:
from pprint import pprint
import re, glob, time
import numpy as np
import pandas as pd
pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows',   120)

import google.genai as genai
import openai
import cohere
import chromadb
import sudachipy
from sudachipy import Dictionary, SplitMode # 日本語Tokenizer
import langchain_core
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer # embedding用
import sklearn
from sklearn.metrics import ndcg_score

print(f"genai={genai.__version__}, openai={openai.__version__}, " + 
      f"cohere={cohere.__version__}, chromadb={chromadb.__version__}, " +
      f"langchain_core={langchain_core.__version__}, ")
print(f"sklearn={sklearn.__version__}")

genai=1.56.0, openai=2.13.0, cohere=5.20.0, chromadb=1.3.7, langchain_core=1.2.2, 
sklearn=1.8.0


#### LLMモデル利用定義

In [80]:
# Geminiモデルを指定
GEMINI_EMBEDDING_MODEL = 'gemini-embedding-001'
#GEMINI_LLM_MODEL      = 'gemini-2.0-flash'  # Limit-Rateが高い
#GEMINI_LLM_MODEL      = 'gemini-2.5-flash'  # それなりに賢い
#GEMINI_LLM_MODEL      = 'gemini-2.5-pro'
#GEMINI_LLM_MODEL      = 'gemini-3-pro-preview'

# OpenAIモデルを指定
OPENAI_EMBEDDING_MODEL_SMALL = 'text-embedding-3-small'
OPENAI_EMBEDDING_MODEL_LARGE = 'text-embedding-3-large'

# Cohereモデルを指定
#COHERE_EMBEDDING_MODEL = 'embed-v4.0'
COHERE_LLM_MODEL       = 'command-a-03-2025'
COHERE_RERANK_MODEL     = 'rerank-v3.5'

# Geminiクライアントを作成
with open('keys/Google_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
GEMINI_CLIENT = genai.Client(api_key=api_key)

# Cohereクライアントを作成
with open('keys/Cohere_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
COHERE_CLIENT = cohere.ClientV2(api_key=api_key)

# OpenAIクライアントを作成
with open('keys/OpenAI_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
OPENAI_CLIENT = openai.OpenAI(api_key=api_key)

# Chromaクライアントを作成
CHROMA_CLIENT = chromadb.EphemeralClient()  # インメモリで作成

# PLaMoクライアントを作成
PLAMO_CLIENT = SentenceTransformer('pfnet/plamo-embedding-1b', trust_remote_code=True)

# ruriクライアントを作成
RURI_CLIENT = SentenceTransformer('cl-nagoya/ruri-v3-310m', trust_remote_code=True)

from enum import Enum, auto
class EmbeddingType(Enum): # モデル切り替え用定数の設定
    GEMINI = auto()
    OPENAI_SMALL = auto()
    OPENAI_LARGE = auto()
#    COHERE = auto()  # 未使用
    PLAMO  = auto()
    RURI   = auto()
#======================================
EMBEDDING_MODEL = EmbeddingType.GEMINI   # モデルを指定
#======================================

No sentence-transformers model found with name pfnet/plamo-embedding-1b. Creating a new one with mean pooling.


#### embedding関数

In [3]:
def get_embeddings(texts):
    """共通のEmbedding関数"""

    # 単一文字列でも対応
    single_input = False
    if isinstance(texts, str):
        texts = [texts]
        single_input = True

    match EMBEDDING_MODEL:

        # Gemini
        case EmbeddingType.GEMINI:
            # Embedding取得
            response = GEMINI_CLIENT.models.embed_content(
                model=GEMINI_EMBEDDING_MODEL,
                contents=texts
            )
            embeddings = [e.values for e in response.embeddings]

        # OpenAI
        case EmbeddingType.OPENAI_SMALL | EmbeddingType.OPENAI_LARGE:
            if EMBEDDING_MODEL == EmbeddingType.OPENAI_LARGE:
                OPENAI_EMBEDDING_MODEL = OPENAI_EMBEDDING_MODEL_LARGE 
            else:
                OPENAI_EMBEDDING_MODEL = OPENAI_EMBEDDING_MODEL_SMALL 
            # Embeddings を取得
            response = OPENAI_CLIENT.embeddings.create(
                model=OPENAI_EMBEDDING_MODEL,
                input=texts
            )
            # レスポンスからベクトル抽出（resp.data は入力ごとの要素を持つ）
            embeddings = [d.embedding for d in response.data]

        # PLaMo
        case EmbeddingType.PLAMO:
            embeddings = PLAMO_CLIENT.encode(
                texts,
                batch_size=16,
                show_progress_bar=False,
                convert_to_numpy=True
            )

        # Ruri
        case EmbeddingType.RURI:
            embeddings = RURI_CLIENT.encode(
                texts, 
                batch_size=16, 
                convert_to_numpy=True, 
                show_progress_bar=False
            )

    # 単一入力だったら一次元のリストを返す
    if single_input:
        return embeddings[0]

    return embeddings

# 確認
print(f"EMBEDDING_MODEL={EMBEDDING_MODEL}")
for texts in ['今日はいい天気です。', ['今日はいい天気です。','明日はどうでしょうか？']]:
    embedded_text = get_embeddings(texts)
    if isinstance(embedded_text, np.ndarray):
        print(f"{type(embedded_text)}, {embedded_text.shape}, {embedded_text.flatten()[:4]} ...")
    else:
        try:
            print(f"{type(embedded_text)}, {len(embedded_text)}, {embedded_text[0][:4]} ...")
        except:
            print(f"{type(embedded_text)}, {len(embedded_text)}, {embedded_text[:4]} ...")

EMBEDDING_MODEL=EmbeddingType.GEMINI
<class 'list'>, 3072, [-0.019278863, -0.0022764413, 0.0021871584, -0.08121658] ...
<class 'list'>, 2, [-0.019278863, -0.0022764413, 0.0021871584, -0.08121658] ...


#### 日本語 Tokenizer

入力文を、形態素解析エンジンでトークン化、必要な品詞のみのトークンのリストを返す  

In [4]:
### すだち
tokenizer = Dictionary(dict='full').create()

from sudachipy import dictionary
print([dictionary.Dictionary(dict='full').lookup('撹拌条件')]) # User_dic確認
print([dictionary.Dictionary(dict='full').lookup('hba1c')])   # User_dic確認

def tokenize(text, splitmode='C', pos=False):

    match splitmode.upper():
        case 'A':
            splitmode = SplitMode.A # 超細かい
        case 'B':
            splitmode = SplitMode.B # 細かい
        case _:
            splitmode = SplitMode.C

    tokens = tokenizer.tokenize(text, splitmode) # Defult = SplitMode.C
    if pos:
        pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    tokens = [t.surface() for t in tokens]
    return tokens

def tokenize_and_filter_by_pos(
    text, 
    target_pos=['名詞','動詞','形容詞','副詞','接頭詞'],
    pos=False
):
    # 入力文から指定された品詞のトークンを抽出
    tokens = list(tokenizer.tokenize(text))
    if pos:
        pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    parts_of_speech = [t.part_of_speech() for t in tokens]                             # 品詞情報を抽出
    is_target_pos = [bool(set(p) & set(target_pos)) and 
                     not (bool(set(p) & set(['動詞'])) and
                          bool(set(p) & set(['非自立', '非自立可能'])))
                                                             for p in parts_of_speech] # 必要品詞で抽出
    #pprint(is_target_pos)
    extracted_tokens = [t.surface() for (t, f) in zip(tokens, is_target_pos) if f]
    return extracted_tokens


pprint(tokenize('① 今日は、良い天気です。', pos=True))
pprint(tokenize_and_filter_by_pos('① 今日は、良い天気です。', pos=True))

[<MorphemeList[
  <Morpheme(撹拌条件, 0:4, (1, 0))>,
]>]
[<MorphemeList[
  <Morpheme(hba1c, 0:5, (1, 46))>,
]>]
[('①', ('名詞', '数詞', '*', '*', '*', '*')),
 (' ', ('空白', '*', '*', '*', '*', '*')),
 ('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
['①', ' ', '今日', 'は', '、', '良い', '天気', 'です', '。']
[('①', ('名詞', '数詞', '*', '*', '*', '*')),
 (' ', ('空白', '*', '*', '*', '*', '*')),
 ('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
['①', '今日', 

#### 材料データ名寄せ関数

In [5]:
# 名寄せデータ
Entity_Resolution_dic = {
    r"f(ab')2": r"f(ab')2コートラテックス",
    r"IgG":     r"IgGコートラテックス",
    r"純水":    r"精製水",
}

#分解パターンの定義
# 正規表現
# 末尾の数量、単位の分離 (省略可)
qty_unit_pattern = re.compile(
    r"""
    \s*[:：]*\s*             # 0個以上の空白、セパレータ、空白
    (?P<qty>[\d.,]+)         # 数量 (数値)
    \s*                      # 0個以上の空白
    (?P<unit>[kKmnμµ]{0,1}[a-zA-Z]+|[日時分秒℃度%]|°[CK]|rpm|mol|min|sec)?  # 単位 (省略可) <= 工夫の余地あり
    $                        # 末尾
    """,
    re.VERBOSE
)

ZEN2HAN = str.maketrans(
    "".join(chr(0xff01 + i) for i in range(94))+"　", # Zen
    "".join(chr(0x21   + i) for i in range(94))+" "   # Han
)

def parse_material(line, trace=False):
    '''
    材料行を分解
    材料と数量単位は、空白またはセパレータ[:：]で分かち書きされている
    以下に分解
    1. 先頭[-*・] (無い場合がある) => 無視
    2. 項番: 例: ①,... (1),...   (無い場合がある) => 無視
    3. 材料: 材料名, 補足
      3.1 材料名 数値を含む場合あり => 変数 mate に格納
      3.2 補足「()（）」で囲まれた補足情報が続く場合もあり => ()含め変数 add に格納
    4. 数量, 単位: 無い場合もあり、 数量と単位に間に空白がある場合と無い場合がある
    　4.1 数量(数値、半角または全角、「,.」を含む) => 変数 qtyに格納　
      4.2 単位((L,g)x(そのまま、k,m,μ))、ignore case) => 変数 unit に格納
    '''
    line = line.strip().replace("　", " ")                 # 前後余白を削除、全角スペースを半角に
    line = re.sub(r'^[\-\*]+\s*', '', line)                # 先頭の "-", "*", "•" を除去
    line = re.sub(r'^\s*(\(?\d+\)?|[①-⑳])\s*', '', line)   # 項番（①, ②, (1), 1. など）を除去
    line = line.translate(ZEN2HAN).strip()                 # 数字記号は半角に
    if trace:
        print(f"## {line}")

    # 1. 正規化：セパレータは「空白 or : or ：」
    #    最後の数量部分を優先的に捉えるため "数量+単位" をパターン化して先に抽出
    qty = ''; unit = ''
    m = qty_unit_pattern.search(line)
    if m:
        qty  = m.group('qty')
        unit = m.group('unit')
        # 数量部分を取り除いて「材料側」だけにする
        line_material = line[:m.start()].rstrip()
    else:
        # 数量・単位なし
        line_material = line.strip()

    # 2. 材料名末尾に続く補足括弧を抽出（複数対応）
    #   材料名内部にある括弧は mate に含める
    add = ''
    add_list = []

    # 補足括弧は、文末方向に連続しているものだけ抽出
    add_pattern = re.compile(r"(\([^)]*\)|（[^）]*）)$")

    tmp = line_material
    while True:
        m = add_pattern.search(tmp)
        if not m:
            break
        add_list.insert(0, m.group(1))  # 先頭に挿入して順序維持
        tmp = tmp[:m.start()].rstrip()

    if add_list:
        add = ''.join(add_list)
        mate = tmp.strip()
    else:
        mate = line_material.strip()

    mate_dic = {'material': mate, 'add': add, 'qty': qty, 'unit': unit}
    if trace:
        print(f"## ## {mate_dic}")
    mate_dic = {k: v for k, v in mate_dic.items() if v != '' and v is not None} # 値が''の要素をdrop
    if trace:
        print(f"## ## {mate_dic}")

    return mate_dic


def parse_and_normalize_materials(material_section, trace=False):
    materials = ''
    lines = material_section.strip().split('\n')
    if trace:
        change = []
    for line in lines:
        line = line.strip()
        if line =='':
            continue
        mate_dic = parse_material(line, trace=trace) # 「材料名 (補足) 数量 単位」に分解
        if mate := mate_dic.get('material', False):  # 材料名があれば、
            normalized_mate = Entity_Resolution_dic.get(mate, mate) # 名寄せ
            mate_dic['material'] =  normalized_mate                 # 名寄せ
            if trace:
                change.append('->') if normalized_mate != mate else change.append('--')            
        materials += ' '.join(list(mate_dic.values())) + '\n'

    if trace:
        for i, (org, chg, nor) in enumerate(zip(lines, change, materials.split('\n')),start=1):
            print(f"{i}, {org} {chg} {nor}")
    return materials

# 動作確認
tests = '''
- ① 純水：20 μL
* (2) エタノール: 5,000mol
精製水 100°K
水:　１０ ml
水  10秒
食塩水(5%) 10ml
- ⑥ IgG（粒径 100 nm）：1 mL
- ⑦ f(ab')2（粒径 200 nm）：1 mL
- ⑥ IgGコートラテックス（粒径 100 nm）：1 mL
'''
pprint(parse_and_normalize_materials(tests, trace=False))

('精製水 20 μL\n'
 'エタノール 5,000 mol\n'
 '精製水 100 °K\n'
 '水 10 ml\n'
 '水 10 秒\n'
 '食塩水 (5%) 10 ml\n'
 'IgGコートラテックス (粒径 100 nm) 1 mL\n'
 "f(ab')2コートラテックス (粒径 200 nm) 1 mL\n"
 'IgGコートラテックス (粒径 100 nm) 1 mL\n')


#### データ読込関数
データを、ノート名・セクションの辞書の形で返す

In [9]:
def read_dataset(doc_dir='notes', trace=False): # -> dic
    """実験ノート(md)の読み込み"""
    indata_section_names = ['目的','材料','実験手順'] # 入力データでのSection名

    # doc_dir以下のすべての .md ファイルを取得
    md_files = glob.glob(f"./{doc_dir}/*.md")

    # レシピを辞書に格納
    notes_dic = {}
    for i, file_path in enumerate(md_files, start=1):
        with open(file_path, "r", encoding="utf-8") as f:
            note = f.read().strip()
            name = re.search(r"^# (.+)$", note, re.MULTILINE).group(1).strip()
            notes_dic[name] = note.strip()
    print(f"\ndoc_dir={doc_dir}, 読みこんだレシピの数: {len(notes_dic)}")
    if trace:
        first_key = list(notes_dic.keys())[0]; print(notes_dic[first_key]) # 最初のレシピ

    # セクションごとに辞書化
    section_pattern = re.compile(r'^## (.*?)\n(.*?)(?=\n## |\Z)', re.MULTILINE | re.DOTALL)
    for i, (name, note) in enumerate(notes_dic.items()):
        if trace:
            print(f"## {name} ====")
        matches = section_pattern.findall(note)  # セクションごとに分離辞書化
        result_dic = {title.strip(): content.strip() for title, content in matches}

        # Section名を標準化
        for (std_name, indata_name) in zip(SECTION_NAMES, indata_section_names):
            if std_name != indata_name:
                result_dic[std_name] = result_dic.pop(indata_name)

        # allの追加
        result_dic['all'] = '\n\n'.join([f"## {s}\n\n{result_dic[s]}" for s in SECTION_NAMES])

        notes_dic[name] = result_dic
    first_key = list(notes_dic.keys())[0]
    print(f"\n辞書化したノートの数: {len(notes_dic)}\nFirst: {first_key}: {notes_dic[first_key].keys()}")
    pprint(notes_dic[first_key])

    return notes_dic

#### Vector Store作成関数
Vector Store では Metadataに、(1)ノート名、(2)セクション名or'all'を付与

In [7]:
from tqdm import tqdm
def setup_collection(notes_dic):
    """
    コレクションのセットアップ
    Arg:
        notes_dic: {'all': [ノート1,ノート2,...], '目的':[ノート1,ノート1,...], ...}
    return:
        collection:    VectorStore
    """

    # collection作成 (すでに在ったら削除)
    collection_name = 'notes'
    try:
        CHROMA_CLIENT.delete_collection(collection_name)
    except:
        pass
    collection = CHROMA_CLIENT.create_collection(
        name = collection_name,
        metadata={'hnsw:space': 'cosine'}  # 距離メトリック = 'cosine'
    )

    names = list(notes_dic.keys())    # ノート名
    n = 10 # 一度に処理するノート数（x section数 = バッチサイズ） token size limit対策
    for i in tqdm(range(0, len(names), n), mininterval=2, ncols=80): # バッチ毎に
        page_contents = []
        metadatas     = []
        ids           = []

        for j in range(i, min(i+n, len(names))): # ノート毎に
            name = names[j]
            note_dic = notes_dic[name]

            for k, s in enumerate(note_dic.keys(), start=1): # section毎に
                doc_id = f"{name}_{s}"
                ids.append(doc_id)    # id: ユニークな文字列
                
                section = f"## {s}:\n{note_dic[s].strip()}"
                page_contents.append(section)
                metadatas.append({'source': name, 'section': s})

        # Embeddingの取得
        doc_embs = get_embeddings(page_contents)

        # ChromaDBへ一括で追加
        collection.add(
            ids=ids,
            embeddings=doc_embs,
            documents=page_contents,
            metadatas=metadatas
        )
    print(f"DBに追加されたChunk(ベクトル)数: {collection.count()}")

    return collection

### データ読込・Stores作成実行

In [10]:
# クライアントと読み込み、コレクションを初期化
SECTION_NAMES = ['目的','材料','手順'] # 正規化時に付与するセクション名
doc_dir = '実験ノート1206'
print(f"## EMBEDDING_MODEL={EMBEDDING_MODEL}")

notes_dic = read_dataset(doc_dir, trace=False)   # データの読み込み

collection = setup_collection(notes_dic) # Vector Storeの作成

## EMBEDDING_MODEL=EmbeddingType.GEMINI

doc_dir=実験ノート1206, 読みこんだレシピの数: 340

辞書化したノートの数: 340
First: 実験ノート ID1-1: dict_keys(['目的', '材料', '手順', 'all'])
{'all': '## 目的\n'
        '\n'
        '現行HbA1c測定試薬（HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）とHbA1c検出抗体1（マウスモノクローナル，HRP標識，全長IgG）、NaOH '
        '10mLとTris '
        '10mL、ラテックス粒径100nmと200nm）の条件を基準条件として設定し、感度・直線性・バックグラウンドなど基本性能を初期評価する。\n'
        '\n'
        '## 材料\n'
        '\n'
        '- ① 純水：20 mL\n'
        '- ② HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）：1 mL\n'
        '- ③ HbA1c検出抗体1（マウスモノクローナル，HRP標識，全長IgG）：1 mL\n'
        '- ④ NaOH：10 mL\n'
        '- ⑤ Tris：10 mL\n'
        '- ⑥ IgGコートラテックス（粒径 100 nm）：1 mL\n'
        "- ⑦ f(ab')2コートラテックス（粒径 200 nm）：1.0 mL\n"
        '\n'
        '## 手順\n'
        '\n'
        '### 手順1\n'
        '\n'
        '1．試薬Aを作成\n'
        '1-1.①～③の分量を測定＆記録して、ビーカーに入れる\n'
        '1-2.スタラーで撹拌する\n'
        '\u3000撹拌条件\n'
        '\u3000\u3000回転数：80±20rpm、撹拌時間：5±2分、撹拌温度：22±3℃\n'
        '1-3.④～⑦の分量を測定＆記録して、ビーカーに入れる\n'
        '1-4.スタラー

100%|███████████████████████████████████████████| 34/34 [01:08<00:00,  2.03s/it]

DBに追加されたChunk(ベクトル)数: 1360


### nDCG算出¶

#### nDCG用CSVデータ読み込み

In [60]:
# クエリ用CSVをDFに読み込み、クエリと関連度に分解
def read_ndcg_test_case(testcase_path='テストケース.csv'):
    # テストケース
    testcase_df = pd.read_csv(testcase_path, header=0)
    testcase_df[['purpose','materials','methods']] = testcase_df.groupby('test_id').ffill().fillna('')[['purpose','materials','methods']]
    # テスト用クエリ
    testquery_df = testcase_df[['test_id','purpose','materials','methods']].copy().drop_duplicates().reset_index(drop=True)

    for row, tid in enumerate(testquery_df['test_id']):
        no, nid, c = tid.split('_')  # test_id = No_Note_Case
        query = notes_dic['実験ノート '+nid]
        if c != '2':
            testquery_df.loc[row, 'purpose']   = query['目的'].strip() # ..._2 は目的なし
        testquery_df.loc[row, 'materials'] = query['材料'].strip()
        testquery_df.loc[row, 'methods']   = query['手順'].strip()

#    display(testquery_df)
    # 関連度
    relevance_df = testcase_df[['test_id','doc_id','gain']].rename(columns={'doc_id': 'name'}).copy()
    relevance_df['name'] = relevance_df['name'].str.strip()
#    display(relevance_df)

    print(f"\n### nDCG用テストデータ読込")
    print(f"### テストID - 関連度(1~0) - Note, relevance_df: len={len(relevance_df)}")
#    display(relevance_df)

    return testquery_df, relevance_df

testquery_df, relevance_df = read_ndcg_test_case(testcase_path='テストケース.csv')

print(f"## testquery_df")
display(testquery_df)
print(f"## relevance_df")
display(relevance_df)


### nDCG用テストデータ読込
### テストID - 関連度(1~0) - Note, relevance_df: len=125
## testquery_df


,test_id,purpose,materials,methods
0,1_ID1-2_1,1-1基準条件からNaOH量を半分に減らし、pH低下による感度変化や反応性低下の有無を評価し...,- ① 純水：20 mL\n- ② HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）...,### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
1,2_ID1-2_2,,- ① 純水：20 mL\n- ② HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）...,### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
2,3_ID2-17_1,Tris濃度が高い条件ではイオン強度やpH緩衝能が過剰となり、ラテックス凝集やバックグラウン...,- ① 精製水：25 mL\n- ② HbA1c捕捉抗体A（マウスモノクローナル，全長IgG...,### 手順1\n\n1.試薬Aの基準調製\n1-1.材料①〜③の投入\n・①～③を順に秤量...
3,4_ID2-17_2,,- ① 精製水：25 mL\n- ② HbA1c捕捉抗体A（マウスモノクローナル，全長IgG...,### 手順1\n\n1.試薬Aの基準調製\n1-1.材料①〜③の投入\n・①～③を順に秤量...
4,5_ID3-30_1,HEPES条件でラテックス粒径を非対称ペアとしタンパク質をBSAへ変更した構成で、感度・直線...,- ① 純水：30 g\n- ② HEPES：1.8 g\n- ③ IgG（粒径 110 n...,### 手順1\n\n1．試薬Aを作成\n1-1.①～④の分量を測定＆記録して、ビーカーに入...
5,6_ID3-30_2,,- ① 純水：30 g\n- ② HEPES：1.8 g\n- ③ IgG（粒径 110 n...,### 手順1\n\n1．試薬Aを作成\n1-1.①～④の分量を測定＆記録して、ビーカーに入...
6,7_ID1-3_1,1-1基準条件からNaOH量を2倍に増量し、高pH条件での感度向上の有無と、バックグラウンド...,- ① 純水：20 mL\n- ② HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）...,### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
7,8_ID1-3_2,,- ① 純水：20 mL\n- ② HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）...,### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
8,9_ID4-50_1,4-21〜4-22 および 4-41〜4-49 の検討結果を踏まえ、NPG/C8–C10 ポ...,- 原料1：ネオペンチルグリコール(NPG) 1.0 mol\n- 原料2：高純度C8–C1...,### 合成手順①\n\n1 L ジャケット付きガラス反応器に原料1〜3を秤量投入し、窒素 ...
9,10_ID4-50_2,,- 原料1：ネオペンチルグリコール(NPG) 1.0 mol\n- 原料2：高純度C8–C1...,### 合成手順①\n\n1 L ジャケット付きガラス反応器に原料1〜3を秤量投入し、窒素 ...


## relevance_df


,test_id,name,gain
0,1_ID1-2_1,実験ノート ID1-2,1.000000
1,1_ID1-2_1,実験ノート ID2-16,0.966667
2,1_ID1-2_1,実験ノート ID1-3,0.866667
3,1_ID1-2_1,実験ノート ID2-21,0.833333
4,1_ID1-2_1,実験ノート ID2-47,0.800000
...,...,...,...
120,10_ID4-50_2,実験ノート ID4-49,0.650000
121,10_ID4-50_2,実験ノート ID4-21,0.650000
122,10_ID4-50_2,実験ノート ID4-9,0.650000
123,10_ID4-50_2,実験ノート ID4-15,0.650000


#### nDCGスコア算出関数

In [61]:
def calc_ndcg(retrieved_df, relevance_df, k, trace=False): # -> ndcg
    # 読み込んだCSVクエリ-想定順位より、レシピデータセットを検索、
    # 検索結果のランクを出力、relevanceとマージしてnDCGスコアを計算
    # TopKのK値で足切りされた料理の順位ランクは0(同率最下位)とする
    ndcgs = []
    name_df = pd.DataFrame({'name': notes_dic.keys()}) # 料理名一覧
    # nCDG@K計算用に各データをマージ
    result_df = name_df.copy() # 料理名一覧
    result_df = pd.merge(result_df, retrieved_df, how='left')                  # 検索順位(TopKまで)をマージ
    result_df = pd.merge(result_df, relevance_df[['name','gain']], how='left') # 関連度をマージ
    result_df = result_df.dropna(subset=["score", "gain"], how="all")          # 両方NaNを削除
    result_df = result_df.fillna(0).sort_values('score', ascending=False).reset_index(drop=True).copy() # 残りを0埋め、ソート

    # nCDG@K (K=TopK)
    ndcg = ndcg_score(result_df['gain' ].to_numpy().reshape(1, -1), 
                      result_df['score'].to_numpy().reshape(1, -1),
                      k=k)
    if trace:
        print(f"## k={TopK}, nDCG Score = {ndcg:.6f}")
        display(result_df)

    return ndcg

#### Case Retrieve(note)

In [62]:
### チャンク(ChromaDB)の類似検索 =============

# Retrive関数 -----------------
def retrieve(queries_dic, k=10, trace=False): 
    # queries_dic: {Section:クエリ, ...} -> [[ノート名, 順位ランク], ...] k個のリストのリスト
    if trace:
        print(f"### Retrieve: query=\n「{queries_dic}」, k={k} ") 

    section = 'all'
    query = queries_dic[section]
    
    # --- クエリをembedding ---
    query_emb = get_embeddings(query)

    # ChromaDBで類似検索（＝retrieval）=======
    results = collection.query(
        query_embeddings=query_emb,
        n_results = 4*k,
        include = ['documents', 'metadatas', 'distances'],
        where = {'section': section}         # metadataでのfilter条件
    )
    retreaved_docs    = results['documents'][0]
    retreaved_sources = [m['source'] for m in results['metadatas'][0]]
    retreaved_dists   = results['distances'][0]
    retreaved_df = pd.DataFrame({
        'docs': retreaved_docs,
        'name': retreaved_sources,
        'dist': retreaved_dists
    })
    if trace:
        display(retreaved_df)

    return retreaved_df[['name','dist']].head(k)

In [63]:
trace = False

TopK = 10

ndcgs_df = pd.DataFrame({'Test_ID': testquery_df['test_id'].values.tolist() + ['#MEAN']}) # ndcgsの入れ物
name_df = pd.DataFrame({'name': notes_dic.keys()}) # 料理名一覧
for emb_type in list(EmbeddingType):
    EMBEDDING_MODEL = emb_type   # モデルを指定
    print(f"\n### Case: retrieve_by_note, EMBEDDING_MODEL={EMBEDDING_MODEL}, TopK = {TopK}")
    collection = setup_collection(notes_dic)   # 次元が変わるため、collection(VectorStore)を再作成
    emb_model = str(EMBEDDING_MODEL).split('.')[1]

    ndcgs = []
    test_ids = []
    for i, row in list(testquery_df.iterrows()):       # テストケース毎に
        queries = [row['purpose'].strip(), row['materials'].strip(), row['methods'].strip()]  # クエリを抽出
        queries_lst = ['\n'.join([s for s in queries if s])] + queries 
        queries_dic = {s: q if q != '' else '' for q, s in zip(queries_lst, ['all','目的','材料','手順'])}
        test_id = row['test_id']
        test_ids.append(test_id)
    
        # 検索実行（個別のクエリで検索）
        retrieved_df = retrieve(queries_dic, k=TopK) # クエリ方法を指定して実行
        retrieved_df['score'] = 1.0 - retrieved_df['dist']   # dist(値小=高ランク)をscore(値大=高ランク)に
        retrieved_df = retrieved_df.drop(columns=['dist']).sort_values('score', ascending=False)
        print(f"\n## Retrieval Result: test_id={test_id}")
        display(retrieved_df)

        ndcg = calc_ndcg(retrieved_df, relevance_df.query("test_id==@test_id"), k=TopK, trace=True)
        ndcgs.append(ndcg)
        print(f"## {test_id}: nDCG Score = {ndcg:.6f}")

    ndcgs_mean = np.array(ndcgs).mean()
    print(f"### Case: retrieve_by_note, EMBEDDING_MODEL={EMBEDDING_MODEL}: nDCG Score Mean = {ndcgs_mean:.6f}")
    wk_df = pd.DataFrame({'Test_ID':test_ids + ['#MEAN'], f"{emb_model} nDCG Score":ndcgs + [ndcgs_mean]})
    ndcgs_df = pd.merge(ndcgs_df, wk_df, how='left')
    display(ndcgs_df)

ndcgs_retrieve_df = ndcgs_df.assign(logic='retrieve')


### Case: retrieve_by_note, EMBEDDING_MODEL=EmbeddingType.GEMINI, TopK = 10


100%|███████████████████████████████████████████| 34/34 [01:08<00:00,  2.02s/it]


DBに追加されたChunk(ベクトル)数: 1360

## Retrieval Result: test_id=1_ID1-2_1


,name,score
0,実験ノート ID1-2,0.972142
1,実験ノート ID1-3,0.956184
2,実験ノート ID1-5,0.954400
3,実験ノート ID2-16,0.947030
4,実験ノート ID2-46,0.942586
5,実験ノート ID2-47,0.941489
6,実験ノート ID1-1,0.941407
7,実験ノート ID2-48,0.939570
8,実験ノート ID1-20,0.934796
9,実験ノート ID1-80,0.934370


## k=10, nDCG Score = 0.771272


...

,name,score
0,実験ノート ID4-50,0.937978
1,実験ノート ID4-41,0.928710
2,実験ノート ID4-48,0.922163
3,実験ノート ID4-15,0.922157
4,実験ノート ID4-44,0.921687
5,実験ノート ID4-21,0.921393
6,実験ノート ID4-9,0.920575
7,実験ノート ID4-45,0.916127
8,実験ノート ID4-43,0.914969
9,実験ノート ID4-2,0.911788


## k=10, nDCG Score = 0.928964


,name,score,gain
0,実験ノート ID4-50,0.937978,1.00
1,実験ノート ID4-41,0.928710,0.70
2,実験ノート ID4-48,0.922163,0.70
3,実験ノート ID4-15,0.922157,0.65
4,実験ノート ID4-44,0.921687,0.75
5,実験ノート ID4-21,0.921393,0.65
6,実験ノート ID4-9,0.920575,0.65
7,実験ノート ID4-45,0.916127,0.80
8,実験ノート ID4-43,0.914969,0.70
9,実験ノート ID4-2,0.911788,0.00


## 10_ID4-50_2: nDCG Score = 0.928964
### Case: retrieve_by_note, EMBEDDING_MODEL=EmbeddingType.RURI: nDCG Score Mean = 0.492422


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score
0,1_ID1-2_1,0.771272,0.394975,0.394975,0.255349,0.552238
1,2_ID1-2_2,0.562039,0.000000,0.000000,0.000000,0.219097
2,3_ID2-17_1,0.711899,0.476282,0.550762,0.405628,0.548477
3,4_ID2-17_2,0.225782,0.304901,0.066182,0.062699,0.358562
4,5_ID3-30_1,0.780918,0.787478,0.795355,0.810859,0.819990
5,6_ID3-30_2,0.174166,0.000000,0.064133,0.000000,0.000000
6,7_ID1-3_1,0.574621,0.390294,0.390294,0.319970,0.454851
7,8_ID1-3_2,0.537990,0.000000,0.000000,0.063333,0.219097
8,9_ID4-50_1,0.821737,0.940911,0.773634,0.803347,0.822942
9,10_ID4-50_2,0.981128,0.732189,0.626052,0.788677,0.928964


#### Case Retrieve(note)-rerank

In [64]:
### チャンク(ChromaDB)の類似検索 =============

# Retrive関数 -----------------
def retrieve_rerank(queries_dic, k=10, trace=False): 
    # queries_dic: {Section:クエリ, ...} -> [[ノート名, 順位ランク], ...] k個のリストのリスト
    if trace:
        print(f"### Retrieve: query=\n「{queries_dic}」, k={k} ") 

    section = 'all'
    query = queries_dic[section]
    
    # --- クエリをembedding ---
    query_emb = get_embeddings(query)

    # ChromaDBで類似検索（＝retrieval）=======
    results = collection.query(
        query_embeddings=query_emb,
        n_results = 4*k,
        include = ['documents', 'metadatas', 'distances'],
        where = {'section': section}         # metadataでのfilter条件
    )
    retreaved_docs    = results['documents'][0]
    retreaved_sources = [m['source'] for m in results['metadatas'][0]]
    retreaved_dists   = results['distances'][0]
    retreaved_df = pd.DataFrame({
        'docs': retreaved_docs,
        'name': retreaved_sources,
        'dist': retreaved_dists
    })
    if trace:
        display(retreaved_df)

    # Cohereでdocumentsをリランク(Rerank) =============
    results = COHERE_CLIENT.rerank(
        model=COHERE_RERANK_MODEL,
        query=query,
        documents=retreaved_docs,
        top_n=4*k,
    ).results

    reranked_df = pd.DataFrame({
         'docs':  [retreaved_docs[r.index]    for r in results],
         'name':  [retreaved_sources[r.index] for r in results],
         'score': [r.relevance_score          for r in results]  # rerank score
     })

    return reranked_df[['name','score']].head(k)

In [65]:
trace = False

TopK = 10

ndcgs_df = pd.DataFrame({'Test_ID': testquery_df['test_id'].values.tolist() + ['#MEAN']}) # ndcgsの入れ物
name_df = pd.DataFrame({'name': notes_dic.keys()}) # 料理名一覧
for emb_type in list(EmbeddingType):
    EMBEDDING_MODEL = emb_type   # モデルを指定
    print(f"\n### Case: retrieve_by_note+rerank, EMBEDDING_MODEL={EMBEDDING_MODEL}, TopK={TopK}")
    collection = setup_collection(notes_dic)   # 次元が変わるため、collection(VectorStore)を再作成
    emb_model = str(EMBEDDING_MODEL).split('.')[1]

    ndcgs = []
    test_ids = []
    for i, row in testquery_df.iterrows():       # テストケース毎に
        queries = [row['purpose'].strip(), row['materials'].strip(), row['methods'].strip()]  # クエリを抽出
        queries_lst = ['\n'.join([s for s in queries if s])] + queries 
        queries_dic = {s: q if q != '' else '' for q, s in zip(queries_lst, ['all','目的','材料','手順'])}
        test_id = row['test_id']
        test_ids.append(test_id)
    
        # 検索実行（個別のクエリで検索）
        retrieved_df = retrieve_rerank(queries_dic, k=TopK) # クエリ方法を指定して実行
        print(f"\n## Retrieval Result: test_id={test_id}")
        display(retrieved_df)

        ndcg = calc_ndcg(retrieved_df, relevance_df.query("test_id==@test_id"), k=TopK, trace=True)
        ndcgs.append(ndcg)
        print(f"## {test_id}: nDCG Score = {ndcg:.6f}")
    
        time.sleep(30)  # リクエストレート制限対策
    
    ndcgs_mean = np.array(ndcgs).mean()
    print(f"### Case: retrieve_by_note, EMBEDDING_MODEL={EMBEDDING_MODEL}: nDCG Score Mean = {ndcgs_mean:.6f}")
    wk_df = pd.DataFrame({'Test_ID':test_ids + ['#MEAN'], f"{emb_model} nDCG Score":ndcgs + [ndcgs_mean]})
    ndcgs_df = pd.merge(ndcgs_df, wk_df, how='left')
    display(ndcgs_df)

ndcgs_rerank_df = ndcgs_df.assign(logic='rerank')


### Case: retrieve_by_note, EMBEDDING_MODEL=EmbeddingType.GEMINI, TopK=10


100%|███████████████████████████████████████████| 34/34 [01:07<00:00,  1.97s/it]


DBに追加されたChunk(ベクトル)数: 1360

## Retrieval Result: test_id=1_ID1-2_1


,name,score
0,実験ノート ID1-2,0.949337
1,実験ノート ID2-16,0.876307
2,実験ノート ID2-46,0.846768
3,実験ノート ID2-47,0.837421
4,実験ノート ID1-5,0.832578
5,実験ノート ID2-50,0.823570
6,実験ノート ID2-21,0.821861
7,実験ノート ID2-9,0.805892
8,実験ノート ID2-48,0.803683
9,実験ノート ID2-25,0.802200


## k=10, nDCG Score = 0.797014


...

,name,score
0,実験ノート ID4-50,0.935400
1,実験ノート ID4-45,0.897633
2,実験ノート ID4-43,0.887761
3,実験ノート ID4-48,0.877571
4,実験ノート ID4-44,0.875287
5,実験ノート ID4-22,0.861643
6,実験ノート ID4-49,0.856249
7,実験ノート ID4-9,0.853925
8,実験ノート ID4-41,0.848885
9,実験ノート ID4-47,0.845241


## k=10, nDCG Score = 0.944608


,name,score,gain
0,実験ノート ID4-50,0.935400,1.00
1,実験ノート ID4-45,0.897633,0.80
2,実験ノート ID4-43,0.887761,0.70
3,実験ノート ID4-48,0.877571,0.70
4,実験ノート ID4-44,0.875287,0.75
5,実験ノート ID4-22,0.861643,0.70
6,実験ノート ID4-49,0.856249,0.65
7,実験ノート ID4-9,0.853925,0.65
8,実験ノート ID4-41,0.848885,0.70
9,実験ノート ID4-47,0.845241,0.00


## 10_ID4-50_2: nDCG Score = 0.944608
### Case: retrieve_by_note, EMBEDDING_MODEL=EmbeddingType.RURI: nDCG Score Mean = 0.535584


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score
0,1_ID1-2_1,0.797014,0.498416,0.444287,0.444287,0.724483
1,2_ID1-2_2,0.654448,0.334411,0.388788,0.548687,0.219097
2,3_ID2-17_1,0.660882,0.660882,0.647564,0.504803,0.720592
3,4_ID2-17_2,0.503972,0.314818,0.489141,0.704566,0.468488
4,5_ID3-30_1,0.780204,0.780204,0.774896,0.718832,0.649962
5,6_ID3-30_2,0.266305,0.000000,0.000000,0.000000,0.000000
6,7_ID1-3_1,0.674593,0.467379,0.642161,0.460846,0.414990
7,8_ID1-3_2,0.523301,0.322879,0.393754,0.541411,0.219097
8,9_ID4-50_1,0.994522,0.994522,0.994522,0.882197,0.994522
9,10_ID4-50_2,0.944608,0.942605,0.942605,0.993961,0.944608


#### Case Retrieve(Parent Chunks) - rerank - aggragate

In [81]:
### チャンク(ChromaDB)の類似検索 =============

# Retrive関数 -----------------
def retrieve_parentChunk(queries_dic, k=10, trace=False): 
    # queries_dic: {Section:クエリ, ...} -> [[ノート名, 順位ランク], ...] k個のリストのリスト
    if trace:
        print(f"### Retrieve: query=\n「{queries_dic}」, k={k} ") 

    # section毎にフィルタをかけてRetrieve, rerank
    reranked_df = pd.DataFrame({
        'docs':   pd.Series(dtype=str),   # 検索された内容
        'source': pd.Series(dtype=str),   # ノート名
        'score':  pd.Series(dtype=float)  # Retricvaでの距離計算結果
    })
    n_queries = 0
    for i, section in enumerate(['all','目的','材料','手順']):
        query = queries_dic[section]
        if query == '':
            continue
        
        n_queries += 1
        # --- クエリをembedding ---
        query_emb = get_embeddings(query)
    
        # ChromaDBで類似検索（＝retrieval）=======
        results = collection.query(
            query_embeddings=query_emb,
            n_results = 4*k,
            include = ['documents', 'metadatas', 'distances'],
            where = {'section': section}         # metadataでのfilter条件
        )
        retreaved_docs    = results['documents'][0]
        retreaved_sources = [m['source'] for m in results['metadatas'][0]]
        retreaved_dists   = results['distances'][0]
        retreaved_df = pd.DataFrame({
            'docs': retreaved_docs,
            'name': retreaved_sources,
            'dist': retreaved_dists
        })
        if trace:
            display(retreaved_df)

        # Cohereでdocumentsをリランク(Rerank) =============
        results = COHERE_CLIENT.rerank(
            model=COHERE_RERANK_MODEL,
            query=query,
            documents=retreaved_docs,
            top_n=4*k,
        ).results

        reranked_df = pd.concat([reranked_df,  # section毎のrerankを連結
                                 pd.DataFrame({
                                     'docs':  [retreaved_docs[r.index]    for r in results],
                                     'name':  [retreaved_sources[r.index] for r in results],
                                     'score': [r.relevance_score          for r in results]  # rerank score
                                 })], axis=0, ignore_index=True)
    aggr_reranked_df = (
        reranked_df.groupby('name', as_index=False)['score'].sum()  # sourceで集約, スコアは合計
        .sort_values('score', ascending=False).reset_index(drop=True) # scoreでソート
    ).head(k)
    aggr_reranked_df['score'] = aggr_reranked_df['score'] / n_queries # Queryの数で割り戻す
    if trace:
        display(reranked_df.sort_values(['score'], ascending=False))
        display(aggr_reranked_df)

    return aggr_reranked_df[['name','score']]

In [82]:
trace = False

TopK = 10
aggr_strategy='MEAN' # スコア集約戦略

ndcgs_df = pd.DataFrame({'Test_ID': testquery_df['test_id'].values.tolist() + ['#MEAN']}) # ndcgsの入れ物
name_df = pd.DataFrame({'name': notes_dic.keys()}) # 料理名一覧
for emb_type in list(EmbeddingType):
    EMBEDDING_MODEL = emb_type   # モデルを指定
    print(f"\n### Case: retrieve_rerank_Parentchunk, EMBEDDING_MODEL={EMBEDDING_MODEL}, TopK = {TopK}")
    collection = setup_collection(notes_dic)   # 次元が変わるため、collection(VectorStore)を再作成
    emb_model = str(EMBEDDING_MODEL).split('.')[1]

    ndcgs = []
    test_ids = []
    for i, row in testquery_df.iterrows():       # テストケース毎に
        queries = [row['purpose'].strip(), row['materials'].strip(), row['methods'].strip()]  # クエリを抽出
        queries_lst = ['\n'.join([s for s in queries if s])] + queries 
        queries_dic = {s: q if q != '' else '' for q, s in zip(queries_lst, ['all','目的','材料','手順'])}
        test_id = row['test_id']
        test_ids.append(test_id)
    
        # 検索実行（個別のクエリで検索）
        retrieved_df = retrieve_parentChunk(queries_dic, k=TopK) # クエリ方法を指定して実行
        print(f"\n## Retrieval Result: test_id={test_id}")
        display(retrieved_df)
    
        ndcg = calc_ndcg(retrieved_df, relevance_df.query("test_id==@test_id"), k=TopK, trace=True)
        ndcgs.append(ndcg)
        print(f"## {test_id}: nDCG Score = {ndcg:.6f}")
    
        time.sleep(30)  # リクエストレート制限対策
    
    ndcgs_mean = np.array(ndcgs).mean()
    print(f"### Case: retrieve_by_note&sections_rerank, EMBEDDING_MODEL={EMBEDDING_MODEL}: nDCG Score Mean = {ndcgs_mean:.6f}")
    wk_df = pd.DataFrame({'Test_ID':test_ids + ['#MEAN'], f"{emb_model} nDCG Score":ndcgs + [ndcgs_mean]})
    ndcgs_df = pd.merge(ndcgs_df, wk_df, how='left')
    display(ndcgs_df)

ndcgs_parentchunk_df = ndcgs_df.assign(logic='parentchunk')


### Case: retrieve_by_note&sections_rerank, EMBEDDING_MODEL=EmbeddingType.GEMINI, TopK = 10


100%|███████████████████████████████████████████| 34/34 [01:09<00:00,  2.04s/it]


DBに追加されたChunk(ベクトル)数: 1360

## Retrieval Result: test_id=1_ID1-2_1


,name,score
0,実験ノート ID1-2,0.717977
1,実験ノート ID1-8,0.691873
2,実験ノート ID1-1,0.662356
3,実験ノート ID2-16,0.631384
4,実験ノート ID1-20,0.622833
5,実験ノート ID1-5,0.617208
6,実験ノート ID1-6,0.588667
7,実験ノート ID1-3,0.582326
8,実験ノート ID2-21,0.575019
9,実験ノート ID2-46,0.557638


## k=10, nDCG Score = 0.645743


...

## k=10, nDCG Score = 0.937071


,name,score,gain
0,実験ノート ID4-50,0.946130,1.00
1,実験ノート ID4-45,0.915872,0.80
2,実験ノート ID4-44,0.892981,0.75
3,実験ノート ID4-43,0.881849,0.70
4,実験ノート ID4-22,0.861743,0.70
5,実験ノート ID4-41,0.858781,0.70
6,実験ノート ID4-9,0.848661,0.65
7,実験ノート ID4-47,0.838669,0.00
8,実験ノート ID4-21,0.818394,0.65
9,実験ノート ID4-42,0.818003,0.65


## 10_ID4-50_2: nDCG Score = 0.937071
### Case: retrieve_by_note&sections_rerank, EMBEDDING_MODEL=EmbeddingType.RURI: nDCG Score Mean = 0.562706


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score
0,1_ID1-2_1,0.645743,0.179786,0.421417,0.321354,0.420129
1,2_ID1-2_2,0.498400,0.065480,0.281580,0.351852,0.424896
2,3_ID2-17_1,0.714762,0.627824,0.638030,0.575457,0.642641
3,4_ID2-17_2,0.711061,0.731496,0.789799,0.726329,0.713449
4,5_ID3-30_1,0.751826,0.627429,0.426346,0.725257,0.538225
5,6_ID3-30_2,0.807062,0.534762,0.382889,0.622956,0.220576
6,7_ID1-3_1,0.535134,0.174337,0.343935,0.327882,0.362384
7,8_ID1-3_2,0.439554,0.065480,0.285763,0.347060,0.429003
8,9_ID4-50_1,0.818944,0.883403,0.818944,0.883403,0.938681
9,10_ID4-50_2,0.995270,0.995270,0.875391,0.995270,0.937071


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score
0,1_ID1-2_1,0.645743,0.179786,0.421417,0.321354,0.420129
1,2_ID1-2_2,0.498400,0.065480,0.281580,0.351852,0.424896
2,3_ID2-17_1,0.714762,0.627824,0.638030,0.575457,0.642641
3,4_ID2-17_2,0.711061,0.731496,0.789799,0.726329,0.713449
4,5_ID3-30_1,0.751826,0.627429,0.426346,0.725257,0.538225
5,6_ID3-30_2,0.807062,0.534762,0.382889,0.622956,0.220576
6,7_ID1-3_1,0.535134,0.174337,0.343935,0.327882,0.362384
7,8_ID1-3_2,0.439554,0.065480,0.285763,0.347060,0.429003
8,9_ID4-50_1,0.818944,0.883403,0.818944,0.883403,0.938681
9,10_ID4-50_2,0.995270,0.995270,0.875391,0.995270,0.937071


### nDCGまとめ

In [83]:
display(ndcgs_retrieve_df)
display(ndcgs_rerank_df)
display(ndcgs_parentchunk_df)

,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score,logic
0,1_ID1-2_1,0.771272,0.394975,0.394975,0.255349,0.552238,retrieve
1,2_ID1-2_2,0.562039,0.000000,0.000000,0.000000,0.219097,retrieve
2,3_ID2-17_1,0.711899,0.476282,0.550762,0.405628,0.548477,retrieve
3,4_ID2-17_2,0.225782,0.304901,0.066182,0.062699,0.358562,retrieve
4,5_ID3-30_1,0.780918,0.787478,0.795355,0.810859,0.819990,retrieve
5,6_ID3-30_2,0.174166,0.000000,0.064133,0.000000,0.000000,retrieve
6,7_ID1-3_1,0.574621,0.390294,0.390294,0.319970,0.454851,retrieve
7,8_ID1-3_2,0.537990,0.000000,0.000000,0.063333,0.219097,retrieve
8,9_ID4-50_1,0.821737,0.940911,0.773634,0.803347,0.822942,retrieve
9,10_ID4-50_2,0.981128,0.732189,0.626052,0.788677,0.928964,retrieve


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score,logic
0,1_ID1-2_1,0.797014,0.498416,0.444287,0.444287,0.724483,rerank
1,2_ID1-2_2,0.654448,0.334411,0.388788,0.548687,0.219097,rerank
2,3_ID2-17_1,0.660882,0.660882,0.647564,0.504803,0.720592,rerank
3,4_ID2-17_2,0.503972,0.314818,0.489141,0.704566,0.468488,rerank
4,5_ID3-30_1,0.780204,0.780204,0.774896,0.718832,0.649962,rerank
5,6_ID3-30_2,0.266305,0.000000,0.000000,0.000000,0.000000,rerank
6,7_ID1-3_1,0.674593,0.467379,0.642161,0.460846,0.414990,rerank
7,8_ID1-3_2,0.523301,0.322879,0.393754,0.541411,0.219097,rerank
8,9_ID4-50_1,0.994522,0.994522,0.994522,0.882197,0.994522,rerank
9,10_ID4-50_2,0.944608,0.942605,0.942605,0.993961,0.944608,rerank


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score,logic
0,1_ID1-2_1,0.645743,0.179786,0.421417,0.321354,0.420129,parentchunk
1,2_ID1-2_2,0.498400,0.065480,0.281580,0.351852,0.424896,parentchunk
2,3_ID2-17_1,0.714762,0.627824,0.638030,0.575457,0.642641,parentchunk
3,4_ID2-17_2,0.711061,0.731496,0.789799,0.726329,0.713449,parentchunk
4,5_ID3-30_1,0.751826,0.627429,0.426346,0.725257,0.538225,parentchunk
5,6_ID3-30_2,0.807062,0.534762,0.382889,0.622956,0.220576,parentchunk
6,7_ID1-3_1,0.535134,0.174337,0.343935,0.327882,0.362384,parentchunk
7,8_ID1-3_2,0.439554,0.065480,0.285763,0.347060,0.429003,parentchunk
8,9_ID4-50_1,0.818944,0.883403,0.818944,0.883403,0.938681,parentchunk
9,10_ID4-50_2,0.995270,0.995270,0.875391,0.995270,0.937071,parentchunk


#### 縦持ちに変換

In [84]:
def convert_to_long(in_df):
    out_df = in_df.melt(
        id_vars=['Test_ID', 'logic'],   # 固定で残す列
        var_name='emb_model',           # 元の列名が入る列名
        value_name='nDCG Score'         # 値の列名
    )
    return out_df


long1_df = convert_to_long(ndcgs_retrieve_df)
long2_df = convert_to_long(ndcgs_rerank_df)
long3_df = convert_to_long(ndcgs_parentchunk_df)

nDCG_long_df = pd.concat([long1_df, long2_df, long3_df], axis=0, ignore_index=True)
nDCG_long_df['emb_model'] = nDCG_long_df['emb_model'].str.replace(' nDCG Score', '', regex=False) # 列名をモデル名にする

display(nDCG_long_df)

# CSVに書き出し
nDCG_long_df.to_csv('Note_nDCG1_long_df.csv', header=True, index=False) # CSV保存

def display_maxrows(df): # 表示行数制限を一時的に回避
    bk_pd_max_rows = pd.options.display.max_rows
    pd.options.display.max_rows = None
    display(df)    
    pd.options.display.max_rows = bk_pd_max_rows
display_maxrows(nDCG_long_df)

,Test_ID,logic,emb_model,nDCG Score
0,1_ID1-2_1,retrieve,GEMINI,0.771272
1,2_ID1-2_2,retrieve,GEMINI,0.562039
2,3_ID2-17_1,retrieve,GEMINI,0.711899
3,4_ID2-17_2,retrieve,GEMINI,0.225782
4,5_ID3-30_1,retrieve,GEMINI,0.780918
...,...,...,...,...
160,7_ID1-3_1,parentchunk,RURI,0.362384
161,8_ID1-3_2,parentchunk,RURI,0.429003
162,9_ID4-50_1,parentchunk,RURI,0.938681
163,10_ID4-50_2,parentchunk,RURI,0.937071


,Test_ID,logic,emb_model,nDCG Score
0,1_ID1-2_1,retrieve,GEMINI,0.771272
1,2_ID1-2_2,retrieve,GEMINI,0.562039
2,3_ID2-17_1,retrieve,GEMINI,0.711899
3,4_ID2-17_2,retrieve,GEMINI,0.225782
4,5_ID3-30_1,retrieve,GEMINI,0.780918
5,6_ID3-30_2,retrieve,GEMINI,0.174166
6,7_ID1-3_1,retrieve,GEMINI,0.574621
7,8_ID1-3_2,retrieve,GEMINI,0.537990
8,9_ID4-50_1,retrieve,GEMINI,0.821737
9,10_ID4-50_2,retrieve,GEMINI,0.981128
